<a href="https://colab.research.google.com/github/kgautam2103/Eventbrite_automation_libraray_public/blob/main/venue_update_address_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This wil authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, timezone
from dateutil import parser
from google.colab import files

#auth token for zoom account admin user
auth_token = ''
prod_url = 'https://www.eventbriteapi.com/v3/'
venues= '/venues/'
counter=1


current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_venue_update_output'
output_csv = current_ts+'_venue_update_output.csv'

sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['venue_id','update_status','response_text'],counter)

#final_ouput_df = pd.DataFrame(columns=['parent_event_id','child_event_id','new_start_date','new_end_date','update_status','response_text' ])
final_output_list=[]

authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/155uRaj3ZbiCNus9ACup5E1BLnUAEqyv0BdgAPhj5lEw/edit#gid=0')

input_sheet = wb.worksheet('inputdata')
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['venue_id'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

for iter,row in inputdata_df.iterrows():
  venue_id = row['venue_id'].strip()
  
  print('venue_id is ' +venue_id)
  update_request_body = {
  "venue": {
    "address": {"address_1": "https://zoom.us/j/9700172269?pwd=Y2xQT0NhT1Rtak16Z3pTaFQ1Y1Q2QT09"}
  }
  }

  update_event_values = json.dumps(update_request_body)
  print(update_event_values)
  update_event_url = prod_url+venues+venue_id+"/"
  update_response = requests.post(update_event_url, data=update_event_values, headers=auth_headers)
  update_status = update_response.status_code
  update_response_text = update_response.text
  print(update_response_text)
  output_value_list=[venue_id,update_status,update_response_text]
  output_sheet.insert_row(output_value_list,counter)
  final_output_list.append(output_value_list)



final_output_df = pd.DataFrame(final_output_list,columns=['venue_id','update_status','response_text' ])
final_output_df.to_csv(output_csv)
files.download(output_csv)
